# Physical Geodesy lab 2

In [1]:
import numpy as np
from scipy import io, integrate, linalg, signal
from scipy.sparse.linalg import eigs

y=np.array([[3854.692 ,3854.692 ,3854.691] , [3852.795, 3852.794, 3852.793], [3854.671, 3854.671, 3854.673], [3852.769, 3852.771, 3852.775], [3849.806, 3849.811, 3849.815],[3852.789, 3852.790, 3852.787], [3849.823, 3849.823, 3849.820]])
t=np.array([[10*3600 + 60*43 + 32],
     [10*3600 + 57*60 + 13],
     [11*3600 +  8*60 + 29],
     [11*3600 + 19*60 + 29],
     [11*3600 + 29*60 + 18],
     [11*3600 + 37*60 + 40],
     [11*3600 + 47*60 + 33]])

In [2]:
sig_y=np.std(y,axis=1,ddof=1)
sig_ym=sig_y/np.sqrt(3)
print(sig_ym)
y_m=np.mean(y,axis=1)
print(y_m)
dy=y_m-y_m[1]
print(dy)
dy=np.r_[dy[0],dy[2:]]
print(dy)
F=np.array([[1,-1,0,0,0,0,0],
     [0, -1, 1, 0, 0, 0, 0],
     [0, -1, 0, 1, 0, 0, 0],
     [0 ,-1 ,0, 0 ,1 ,0 ,0],
     [0 ,-1 ,0, 0, 0, 1, 0],
     [0 ,-1, 0 ,0 ,0, 0, 1]])
Sig_ym=np.diag(sig_ym**2)
Sig_dy=F@Sig_ym@F.T
sig_dy=np.sqrt(np.diag(Sig_dy))
print(sig_dy*1000)

[0.00033333 0.00057735 0.00066667 0.00176383 0.00260342 0.00088192
 0.001     ]
[3854.69166667 3852.794      3854.67166667 3852.77166667 3849.81066667
 3852.78866667 3849.822     ]
[ 1.89766667  0.          1.87766667 -0.02233333 -2.98333333 -0.00533333
 -2.972     ]
[ 1.89766667  1.87766667 -0.02233333 -2.98333333 -0.00533333 -2.972     ]
[0.66666667 0.8819171  1.85592145 2.66666667 1.05409255 1.15470054]


In [5]:
a=np.array([[-1,0],[-1,0],[0,0],[0,1],[0,0],[0,1]])
dt=(t-t[1]).T
dt=np.r_[dt[0,0],dt[0,2:]]
A=np.c_[a,np.array([dt]).T]
P=1/(sig_dy**2)
P=np.diag(P)
xd=linalg.inv(A.T@P@A)@A.T@P@dy
print(xd)
Qxd=linalg.inv(A.T@P@A)
print(Qxd)
e=dy-A@xd
print(e)
Sig_d=e@P@e.T/(6-3)
print(Sig_d)
Sig_xd=Sig_d*Qxd
sig3=np.sqrt(Sig_xd[2,2])
print(sig3)


[-1.88885590e+00 -2.95796015e+00 -5.55979719e-06]
[[ 2.92645774e-07  1.01040331e-07 -3.54887935e-11]
 [ 1.01040331e-07  2.16270088e-06 -3.65246020e-10]
 [-3.54887935e-11 -3.65246020e-10  1.28286799e-13]]
[ 0.00424618 -0.00743081 -0.01490544 -0.01467058  0.00816029  0.00275073]
90.64490373811367
3.4100651774973968e-06


In [15]:
dh1=6.897
dh2=17.260
sig_h=0.03

dgh1=xd[0]/dh1
dgh2=xd[1]/(dh2-dh1)
print(dgh1)
print(dgh2)
F=np.array([[1/dh1,0,-xd[0]/dh1**2,0],[0,1/dh2,xd[1]/(dh2-dh1)**2,-xd[1]/(dh2-dh1)**2]])
print(F)
Sig_l=np.diag(np.array([Sig_xd[0,0],Sig_xd[1,1],sig_h,sig_h])**2)
Sig_dgdh=F@Sig_l@F.T
print(np.sqrt(np.diag(Sig_dgdh)))

-0.27386630380117627
-0.28543473374900413
[[ 0.14499058  0.          0.03970803  0.        ]
 [ 0.          0.05793743 -0.02754364  0.02754364]]
[0.00119125 0.00116863]
